In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

%matplotlib inline
import warnings

In [2]:
#loading data
dataInfo = pd.read_csv('DataDictionary.csv')
policy = pd.read_csv('policies.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
samplesubmission = pd.read_csv('SampleSubmission.csv')

In [3]:
#viewing the data
dataInfo.head()

,Policy Data,Unnamed: 1
0,Columns,Data type
1,policy number,Unique policy identifier
2,product code,product code
3,trans date,date of policy transaction
4,from date,first date of policy cover


In [4]:
# Convert datetime variables to datetime objects
policy.from_date = pd.to_datetime(policy.from_date, errors='coerce')
policy.trans_date = pd.to_datetime(policy.trans_date, errors='coerce')


# Confirm that the dates have been converted successfully 
policy.from_date.describe()

<ipython-input-4-8c1230b03af9>:7: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  policy.from_date.describe()


count                   30371
unique                   1431
top       2018-02-01 00:00:00
freq                     1122
first     2017-01-01 00:00:00
last      2021-11-17 00:00:00
Name: from_date, dtype: object

In [9]:
# Extract day, month and year from the loss date column
# day
policy['trans_date_day'] =policy.trans_date.dt.month

# month
policy['trans_date_month'] = policy.trans_date.dt.month
 
# year
policy['trans_date_year'] = policy.trans_date.dt.year
# day
policy['from_date_day'] =policy.from_date.dt.day

# month
policy['from_date_month'] = policy.from_date.dt.month
 
# year
policy['from_date_year'] = policy.from_date.dt.year

In [13]:
# Convert categorical variables to categorical objects
cat_cols = policy.columns.difference(['from_date_year', 'from_date_month', 'from_date_day','trans_date_year','trans_date_month','trans_date_day',' trans_date',' from_date']).tolist()

# Preview categorical variables
policy[cat_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30371 entries, 0 to 30370
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   agent              9376 non-null   category      
 1   branch             9376 non-null   category      
 2   class_of_business  9376 non-null   category      
 3   client_type        9376 non-null   category      
 4   from_date          30371 non-null  datetime64[ns]
 5   policy_number      30371 non-null  object        
 6   product_code       30371 non-null  object        
 7   renewal_frequency  30371 non-null  object        
 8   to_date            30371 non-null  object        
 9   trans_date         30371 non-null  datetime64[ns]
dtypes: category(4), datetime64[ns](2), object(4)
memory usage: 1.0+ MB


In [16]:
policy[cat_cols] = policy[cat_cols].apply(lambda x: x.fillna(x.value_counts().index[0]))

In [17]:
for col in cat_cols:
  policy[col] = train[col].astype('category')

# Confirm
policy.info()

KeyError: 'from_date'

In [6]:
test = pd.merge(policy,test)

In [7]:
test

,policy_number,product_code,trans_date,from_date,to_date,agent,class_of_business,client_type,renewal_frequency,branch,claim_id,claim_number,sum_insured,product,risk_type,loss_date,primary_cause,secondary_cause
0,pol000009828,prod00005,2017-07-01 09:04:02.217,2017-07-03,2017-11-02 00:00:00.000,ag00226,cob00009,ct0003,rf0001,br00006,ID_IKA9OJWSZMF7,claim0011437,4500.0,prod00005,rt00032,2018-10-15,pc0007,sc00024
1,pol000009828,prod00005,2017-10-31 14:09:51.603,2017-11-03,2018-03-02 00:00:00.000,ag00226,cob00009,ct0003,rf0001,br00006,ID_IKA9OJWSZMF7,claim0011437,4500.0,prod00005,rt00032,2018-10-15,pc0007,sc00024
2,pol000009828,prod00005,2018-03-05 17:08:34.343,2018-03-05,2018-07-04 00:00:00.000,ag00226,cob00009,ct0003,rf0001,br00006,ID_IKA9OJWSZMF7,claim0011437,4500.0,prod00005,rt00032,2018-10-15,pc0007,sc00024
3,pol000009828,prod00005,2018-07-04 16:21:28.697,2018-07-05,2018-11-04 00:00:00.000,ag00226,cob00009,ct0003,rf0001,br00006,ID_IKA9OJWSZMF7,claim0011437,4500.0,prod00005,rt00032,2018-10-15,pc0007,sc00024
4,pol000008176,prod00005,2017-07-04 13:08:10.133,2017-07-04,2017-11-03 00:00:00.000,ag00226,cob00009,ct0003,rf0001,br00001,ID_L3A7EFUIWUTO,claim0008881,7000.0,prod00005,rt00032,2018-04-27,pc0007,sc00023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4303,pol000009944,prod00027,2021-06-08 13:04:38.710,2021-05-21,2021-11-10 00:00:00.000,ag00080,cob00031,ct0003,rf0004,br00008,ID_OBGVIJBTYGGN,claim0011625,1190000.0,prod00027,rt00006,2021-06-04,pc0007,sc00022
4304,pol000009960,prod00015,2021-06-29 15:14:52.750,2021-06-01,2021-11-30 00:00:00.000,ag00226,cob00034,ct0002,rf0004,br00008,ID_V3O6J6UXRJ68,claim0011719,7600000.0,prod00015,rt00023,2021-07-07,pc0007,sc00031
4305,pol000009960,prod00015,2021-06-29 15:14:52.750,2021-06-01,2021-11-30 00:00:00.000,ag00226,cob00034,ct0002,rf0004,br00008,ID_PC5P7DB29VI4,claim0011987,7600000.0,prod00015,rt00023,2021-06-22,pc0007,sc00024
4306,pol000010724,prod00027,2021-08-02 13:23:41.613,2021-02-22,2022-02-21 00:00:00.000,ag00044,cob00031,ct0003,rf0002,br00009,ID_JBXG583556E0,claim0013073,450000.0,prod00027,rt00006,2021-07-11,pc0007,sc00022
